# Sequence to Sequence Learning with Neural Networks (NIPS 2014) 

from IPython.display import YouTubeVideo
YouTubeVideo('https://youtu.be/VxR0sQUFKUc')

![Model structer](https://velog.velcdn.com/images%2Fxuio%2Fpost%2Fdd42208d-e3b1-4719-a198-c01efb6adc44%2F%E1%84%89%E1%85%B3%E1%84%8F%E1%85%B3%E1%84%85%E1%85%B5%E1%86%AB%E1%84%89%E1%85%A3%E1%86%BA%202022-01-28%20%E1%84%8B%E1%85%A9%E1%84%8C%E1%85%A5%E1%86%AB%2011.05.09.png)

- 'Sequence to Sequence는 하나의 Sequence에서 다른 Sequence로 번역을 진행 하겠다.'라는 것이다.

기존에는 RNN기반의 번역을 진행하였는데 한계점이 많이 존재 했었다. 하지만 Sequence to Sequence의 논문이 나오면서 급격하게 딥러닝 기반 번역에 대한 기술이 많이 발전하게 되었고 Sequence to Sequence이후에 Attention과 transfomer 논문이 나오면서 현재는 transformer 가 state of the art로 사용되고 있는 상황이다.

RNN 기반일 경우에는 입력 데이터와 출력 데이터의 크기가 같아야 번역이 가능했기에 짧은 문장은 번역이 가능했지만 긴 문장은 번역이 불가능 했었다. 하지만 Sequence to Sequence로 오면서 긴 문장도 번역이 가능해졌다.

(RNN은 단어가 입력될 때 마다 hidden state가 바뀌지만 Sequence to Sequenc는 마지막 hidden state만 참고하는 것으로 알고있다.)

sequence는 일반적으로 하나의 문장을 의미한다. 하나의 문장에는 여러개의 단어로 구성되어있고 각각의 단어를 토큰으로써 처리한다.

(Encoder는 번역 전 문장 Sorce sentece Decoder는 번역 후 문장 Target sentece)

이 Sequence to Sequence Learning with Neural Networks의 논문의 특징으로는 **고정된 벡터(Fiexed Vector)** 즉 Vector의 크기가 고정되어 있다는 점이다. 고정된 벡터에는 어떠한 문장의 정보를 모두 담아야 한다는 것이다.

그렇기 때문에 고정된 크기 Vector에서 긴 문장만 입력되다가 짧은 문장이 나오면 효율이 감소하는 경우가 발생하게 된다.

그리고 seq2seq의 아키텍처에 RNN을 적용하는것 보다 LSTM을 적용했을때 빈약적으로 성능이 향상 된것을 확인 할 수 있다. 또한 일반적인 방법으로 즉 reverse를 진행하지 않고 번역을 진행 했을때는 sorce sectece와 target sentece의 상응 하는 단어의 평균 distance멀어 'minimal time lag'이 발생하는데 reverse를 했을때 평균 distance가 변하지는 않지만 source language 안에 상응 하는 첫 몇몇의 단어가 target language의 첫 몇몇의 단어와 가까워져 'minimal time lag'가 매우 줄어 들었다는 것을 확인 할 수 있었다.

딥러닝을 할때 임베딩(Embedding)을 하는 이유는 원-핫 인코딩(one-hot encoding)시 차원의 크기가 매우 커지기 때문에 작은 데이터로 표현될 수 있도록 임베딩 레이어를 이용한다.

sos -> 시퀀스 시작을 알리기 위해

eos -> 시퀀스 끝을 알리기 위해

## 데이터 전처리(Preprocessing)

- spacy 라이브러리: 문장의 토큰화(tokenization), 태깅(tagging) 등의 전처리 기능을 위한 라이브러리
  - 영어(Engilsh)와 독일어(Deutsch) 전처리 모듈 설치

In [4]:
import torch
import torchtext
import torch.optim as optim
import torch.nn as nn

from torchtext.legacy.data import BucketIterator, Field
from torchtext.legacy.datasets import Multi30k

import spacy
import numpy as np

import random
import time
import math

In [5]:
!python -m spacy download en_core_web_sm
!python -m spacy download de_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 14.9 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 28.0 MB/s eta 0:00:0000:0100:01


✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')


In [6]:
spacy_de = spacy.load('de_core_news_sm') # 영어 토큰화(tokenization)
spacy_en = spacy.load('en_core_web_sm') # 독일어 토큰화(tokenization)

- 영어(English) 및 독일어(Deuthsch) **토큰화 함수** 정의

In [7]:
# 독일어(Deuthsch) 문장을 토큰화한 뒤에 순서를 뒤집는 함수
def tokenize_de(text):
    
    return [token.text for token in spacy_de.tokenizer(text)][::-1]

# 영어(English) 문장을 토큰화 하는 함수
def tokenize_en(text):
    return [token.text for token in spacy_en.tokenizer(text)]

- **필드(Field)** 라이브러리를 이용해 데이터세에 대한 구체적인 전처리 내용을 명시한다.
- 번역 목표 = SRC(독일어), TRG(영어)

In [8]:
SRC = Field(tokenize=tokenize_de,init_token="<sos>",eos_token="<eos>",lower=True)

TRG = Field(tokenize=tokenize_en,init_token="<sos>",eos_token="<eos>",lower=True)

- 대표적인 영어-독어 번역 데이터셋인  **Multi30k**를 불러온다.

In [9]:
train_data, valid_data, test_data = Multi30k.splits(exts= ('.de', '.en'),fields = (SRC, TRG))

In [10]:
print(f"학습 데이터셋(training dataset) 크기: {len(train_data.examples)}")
print(f"평가 데이터셋(validation dataset) 크기: {len(valid_data.examples)}")
print(f"테스트 데이터셋(testing dataset) 크기: {len(test_data.examples)}")

학습 데이터셋(training dataset) 크기: 29000
평가 데이터셋(validation dataset) 크기: 1014
테스트 데이터셋(testing dataset) 크기: 1000


In [11]:
print(vars(train_data.examples[0]))

{'src': ['.', 'büsche', 'vieler', 'nähe', 'der', 'in', 'freien', 'im', 'sind', 'männer', 'weiße', 'junge', 'zwei'], 'trg': ['two', 'young', ',', 'white', 'males', 'are', 'outside', 'near', 'many', 'bushes', '.']}


- **필드(Field)** 객체의 **build_vocab** 메서드를 이용해 영어와 독어의 단어 사전을 생성
- **최소 2번이상** 등장한 단어만을 선택한다.

In [12]:
# 학습 데이터 중 하나를 선택해 출력
print(vars(train_data.examples[30])['src'])
print(vars(train_data.examples[30])['trg'])

['.', 'steht', 'urinal', 'einem', 'an', 'kaffee', 'tasse', 'einer', 'mit', 'der', ',', 'mann', 'ein']
['a', 'man', 'standing', 'at', 'a', 'urinal', 'with', 'a', 'coffee', 'cup', '.']


In [13]:
SRC.build_vocab(train_data, min_freq =2)
TRG.build_vocab(train_data, min_freq =2)

print(f"len(SRC): {len(SRC.vocab)}")
print(f"len(TRG): {len(TRG.vocab)}")

len(SRC): 7853
len(TRG): 5893


In [14]:
print(TRG.vocab.stoi["abcbc"])# 없는 단어 
print(TRG.vocab.stoi[TRG.pad_token]) # 패딩(padding)
print(TRG.vocab.stoi["<sos>"]) # <sos>
print(TRG.vocab.stoi["<eos>"]) # <eos>
print(TRG.vocab.stoi["hello"])
print(TRG.vocab.stoi["world"])

0
1
2
3
4112
1752


- 한 문장에 포함된 단어가 연속적으로 **LSTM**에 입력되어야 한다.
  - 따라서 하나의 배치에 포함된 문장들이 가지는 단어의 개수가 유사하도록 만들면 좋다
  - 이를 위해BucketIterator를 사용한다.
  - **배치 크기(batch size): 128**

In [15]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

BATCH_SIZE = 128

#일반적인 데이터 로더(data loader)의 iterator와 유사하게 사용가능
train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size=BATCH_SIZE,
    device=device)

## 인코더(Encoder)

- 주어진 소스 문장을 **문맥 벡터(context vector)** 로 인코딩 한다.
- LSTM은 hidden state과 cell state을 반환한다.
- forward은 오직 hidden 과 sell을 반환
- 하이버 파라미터(hyperparameter)
  - **input_dim**: 하나의 단어에 대한 원핫 인코딩 차원
  - **embed_dim**: 임베딩(embedding) 차원
  - **hidden_dim**: 히든 상태(hidden state) 차원
  - **n_layers**: RNN 레이어의 개수
  - **dropout_ratio**: 드롭아웃(dropout) 비율

In [16]:
# 인코더 정의

class Encoder(nn.Module):
    def __init__(self, input_dim, embed_dim, hidden_dim, n_layers, dropout_ratio):
        super().__init__()
        
        #임베딩은 원-핫 인코딩을 특정 차원의 임베딩으로 매핑하는 레이어
        self.embedding = nn.Embedding(input_dim, embed_dim)
        
        #LSTM 레이어
        self.hidden_dim = hidden_dim
        self.n_layers  = n_layers
        self.rnn = nn.LSTM(embed_dim, hidden_dim, n_layers, dropout = dropout_ratio)
        
        #드롭아웃(dropout)
        self.dropout = nn.Dropout(dropout_ratio)
        
    #인코더는 소스 문장을 입력으로 받아 문맥 벡터(context vector)를 반환
    def forward(self, src):
        embedded = self.dropout(self.embedding(src))
        # embedded : [단어 개수, 배치 크기, 임베딩 차원]
        
        outputs, (hidden, cell) = self.rnn(embedded)
        #outputs: [단어 개수, 배치 크기, 히든 차원] : 현재 단어의 출력 정보
        #hidden: [레이어 개수, 배치 크기, 히든 차원]: 현재까지의 모든 단어의 정보
        #cell: [레이어 개수, 배치 크기, 히든 차원]: 현재까지의 모든 단어의 정보
        
        #문맥 벡터(context vector)반환
        return hidden, cell

## 디코더(Decoder)

- 주어진 문맥 백터(context vector)를 타켓 문장으로 디코딩 한다.
- LSTM은 hidden state와 cell state를 반환한다.
- 하이버 파라미터(hyperparameter)
  - **input_dim**: 하나의 단어에 대한 원핫 인코딩 차원
  - **embed_dim**: 임베딩(embedding) 차원
  - **hidden_dim**: 히든 상태(hidden state) 차원
  - **n_layers**: RNN 레이어의 개수
  - **dropout_ratio**: 드롭아웃(dropout) 비율

In [36]:
# 디코더 정의

class Decoder(nn.Module):
    def __init__(self, output_dim, embed_dim, hidden_dim, n_layers, dropout_ratio):
        super().__init__()
        
        #임베딩은 원-핫 인코딩말고 특정 차원의 임베딩으로 매핑하는 레이어
        self.embedding = nn.Embedding(output_dim, embed_dim)
        
        #LSTM 레이어
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.rnn = nn.LSTM(embed_dim, hidden_dim, n_layers, dropout = dropout_ratio)
        
        #FC레이어 (인코더와 구조적으로 다른 부분)
        self.output_dim = output_dim
        self.fc_out = nn.Linear(hidden_dim, output_dim)
        
        #드롭아웃(dropout)
        self.dropout = nn.Dropout(dropout_ratio)
        
    # 디코더는 현재까지 출력된 문장에 대한 정보를 입력으로 받아 타켓 문장을 반환
    def forward(self, input, hidden, cell):
        # input: [배치 크기]: 단어의 개수는 항상 1개이도록 구현
        # hidden: [레이어 개수, 배치 크기, 히든 차원]
        # cell = context: [레이어 개수, 배치 크기, 히든 차원]
        input = input.unsqueeze(0)
        # input: [단어의 개수 = 1. 배치 크기]
        
        embedded = self.dropout(self.embedding(input))
        #embedded: [단어의 개수, 배치 크기. 임베딩 차원]
        
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        # output: [단어 개수 = 1, 배치 크기, 히든 차원]: 현재 단어의 출력 정보
        # hidden: [레이어 개수, 배치 크기, 히든 차원]: 현재까지의 모든 단어의 정보
        # cell: [레이어 개수, 배치 크기, 히든 차원]: 현재까지의 모든 단어의 정보
        
        #단어 개수는 어차피 1개이므로 차원 제거
        prediction = self.fc_out(output.squeeze(0))
        #prediction = [배치 크기, 출력 차원]
        
        #(현재 출력 단어, 현재까지의 모든 단어의 정보, 현재까지의 모든 단어의 정보)
        return prediction, hidden, cell

## seq2seq

- 앞서 정의한 인코더()와 디코더()를 가지고 있는 하나의 아키텍처
  - input/source sentece를 받는다.
  - **인코더(encoder)**: 주어진 소스 문장을 문맥 벡터(context vector)로 인코딩 한다.
  - **디코더(decoder)**: 주어진 문맥 벡터(context vector)를 타겟으로 디코딩 한다.
  - 단, **디코더는 한 단어씩** 넣어서 한 번씩 결과를 구한다.
  
- **Teacher forcing**: 디코터의 예측을 다음 입력으로 사용하지 않고, 실제 목표 출력(ground-truth)을 다음 입력으로 사용하는 기법

In [18]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
    #학습할 때는 완전한 형테의 소스 문장, 타겟 문장, teacher_forching_ratio를 넣기
    def forward(self, src, trg, teacher_forcing_ratio= 0.5):
        # src: [단어 개수, 배치 크기]
        # trg: [단어 개수, 배치 크기]
        # 먼저 인코더를 거쳐 문맥 벡터(context vector)를 추출
        hidden, cell = self.encoder(src)
        
        # 디코더(decoder)의 최종 결과를 담을 텐서 캑체 만들기
        trg_len = trg.shape[0] # 단어의 갯수
        batch_size = trg.shape[1] # 배치의 크기
        trg_vocab_size = self.decoder.output_dim # 출력 차원
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        # 첫 번째 입력은 항상 <sos> 토큰
        input = trg[0, :]
        
        #타겟 단어의 개수만큼 반복하여 디코더에 포워딩(forwarding)
        for t in range(1, trg_len):
            output, hidden, cell = self.decoder(input, hidden, cell)
            
            outputs[t] = output #FC를 거쳐서 나온 현재의 출력 단어 정보
            top1 = output.argmax(1) # 가장 확률이 높은 단어의 인데스 추출
            
            # teacher_forcing_ratio: 학습할 때 실제 목표 출력(ground-truth)을 사용하는 비율
            teacher_force = random.random() < teacher_forcing_ratio
            input = trg[t] if teacher_force else top1 # 현재의 출력 결과를 다음 입력에서 넣기
            
        return outputs

### 학습(Training)

 - 하이퍼 파라미터 설정 및 모델 초기화

In [19]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENCODER_EMBED_DIM = 256
DECODER_EMBED_DIM = 256
HIDDEN_DIM = 512
N_LAYERS =2
ENC_DROPOUT_RATIO = 0.5
DEC_DROPOUT_RATIO = 0.5

In [20]:
#인코더와 디코더 객체 선언
enc = Encoder(INPUT_DIM, ENCODER_EMBED_DIM, HIDDEN_DIM, N_LAYERS, ENC_DROPOUT_RATIO)
dec = Decoder(OUTPUT_DIM, DECODER_EMBED_DIM, HIDDEN_DIM, N_LAYERS, DEC_DROPOUT_RATIO)

# Seq2Seq 객체 선언
model = Seq2Seq(enc, dec, device).to(device)

- 논문의 내용대로(−0.08,0.08)의 값을로 **모델 가중치 파라미터 초기화**

In [21]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(7853, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(5893, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=512, out_features=5893, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

###  학습 및 평가 함수 정의

In [22]:
# Adam optimizer로 학습 최적화
optimizer = optim.Adam(model.parameters())

# 뒷 부분의 패딩(padding)에 대해서는 값 무시
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]
criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [23]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trinable parameters')

The model has 13,898,501 trinable parameters


In [24]:
#모델 학습(train) 함수
def train(model, iterator, optimizer, ctiterion, clip):
    model.train() # 학습 모드
    epoch_loss=0
    
    #전체 학습 데이터를 확인하며
    for i, batch in enumerate(iterator):
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        # output: [출력 단어 개수, 배치 크기. 출력 차원]
        output_dim = output.shape[-1]
        
        # 출력 단어의 인덱스 0은 사용하지 않음
        output = output[1:].view(-1, output_dim)
        #output = [(출력 단어의 개수 -1) * batch size, output dim]
        trg = trg[1:].view(-1)
        #trg = [(타겟 단어의 개수 -1) * batch size]
        
        #모델의 출력 결과와 타겟 문장을 비교하여 손실 계산
        loss = criterion(output, trg)
        loss.backward() # 기울기(gradient) 계산
        
        #기울기(gradient) clipping진행
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
            
        #파라미터 업데이트
        optimizer.step()
        
        #전체 손실 값 계산
        epoch_loss += loss.item()
        
        
    return epoch_loss / len(iterator)

with torch.no_grad(): block 안에서 기울기(graident)를 계산하지 않게 한다. 메모리 소비를 줄이고 스피드를 빠르게 해준다

In [25]:
# 모델 평가(evaluate) 함수
def evaluate(model, iterator, criterion):
    model.eval() #평가 모드 (dropout turn off)
    epoch_loss = 0
    
    with torch.no_grad(): 
        #전체 평가 데이터를 확인하며
        for i, batch in enumerate(iterator):
            src = batch.src
            trg = batch.trg
            
            #평가할 때 teacher forcing는 사용하지 않음
            output = model(src, trg, 0)
            #output: [출력 단어 개수, 배치 크기. 출력 차원]
            output_dim = output.shape[-1]
            
            #출력 단어의 인덱스 0은 사용하지 않음
            output = output[1:].view(-1, output_dim)
            #output = [(출력 단어의 개수 -1) * batch size.output_dim]
            trg = trg[1:].view(-1)
            #trg = [(타겟 단어의 개수 -1) * batch size]
            
            #모델의 출력 결과와 타겟 문장을 비교하여 손실 계산
            loss = criterion(output, trg)
            
            #전체 손실 삾 계산
            epoch_loss += loss.item()
            
            
    return epoch_loss / len(iterator)

- 학습(training)및 검증(validation)진행
 - 학습 횟수(epoch): 

In [26]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_time * 60))
    return elapsed_mins, elapsed_secs

펄플렉서티(Perplexity, PPL) = 모델 내에서 자신의 성능을 수치화하여 결과를 내놓는 것이다. 언어의 모델을 평가하기 위한 지표. 성능의 수치가 낮을수록 언어모델의 성능이 좋다는 것을 의미

In [27]:
N_EPOCHS = 15
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    start_time = time.time() # 시작 시간 기록
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time() # 종료 시간 기록
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'seq2seq.pt')
        
        
    print(f'Epoch: {epoch + 1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):.3f}')
    print(f'\tValidation Loss: {valid_loss:.3f} | Validation PPL: {math.exp(valid_loss):.3f}')

Epoch: 01 | Time: 0m -1574s
	Train Loss: 5.054 | Train PPL: 156.688
	Validation Loss: 4.940 | Validation PPL: 139.759
Epoch: 02 | Time: 0m -1590s
	Train Loss: 4.500 | Train PPL: 90.021
	Validation Loss: 4.910 | Validation PPL: 135.604
Epoch: 03 | Time: 0m -1453s
	Train Loss: 4.256 | Train PPL: 70.532
	Validation Loss: 4.724 | Validation PPL: 112.595
Epoch: 04 | Time: 0m -1442s
	Train Loss: 4.025 | Train PPL: 55.973
	Validation Loss: 4.580 | Validation PPL: 97.543
Epoch: 05 | Time: 0m -1531s
	Train Loss: 3.880 | Train PPL: 48.432
	Validation Loss: 4.385 | Validation PPL: 80.218
Epoch: 06 | Time: 0m -1545s
	Train Loss: 3.744 | Train PPL: 42.260
	Validation Loss: 4.309 | Validation PPL: 74.389
Epoch: 07 | Time: 0m -1516s
	Train Loss: 3.619 | Train PPL: 37.318
	Validation Loss: 4.219 | Validation PPL: 67.990
Epoch: 08 | Time: 0m -1553s
	Train Loss: 3.498 | Train PPL: 33.057
	Validation Loss: 4.173 | Validation PPL: 64.930
Epoch: 09 | Time: 0m -1562s
	Train Loss: 3.384 | Train PPL: 29.488
	

In [32]:
model.load_state_dict(torch.load('seq2seq.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 3.905 | Test PPL:  49.655 |


## 나만의 데이터로 모델 사용해보기

In [46]:
#번역 함수
def translate_sentence(sentence, src_field, trg_field, model, divice, max_len_50):
    model,eval()#평가모드
    
    if isinstance(sentecnem, str):
        nip = spacy.load('de')
        tokens = [token.text.lower() for token in nlp(sentecve)]
    else:
        toekns = [token.lower() for token in sentence]
        
    tokens = [src_field.init_token] + tokens + [src_field.eos_token]
    print(f"전체 소스 토큰: {tokens}")
    
    src_tensor = torch.LongTensor(src_indexes).unsqueeze(1).to(device)
    
    with torch.no_grad():
        hidden, cell = model.encoder(src_tensor)
        
    trg_indexes = [trg_field.vocab.stoi[trg_field.init_token]]
    
    for i in range(max_len):
        
        trg_tensor =torch.LongTensor([trg_indexes[-1]]).to(device)
        
        with torch.no_grad():
            output, hidden, cell = model.decoder(trg_tensor, hidden, cell)
        
        pred_token = output.argmax(1).item()
        trg_indexes,append(pred_token)
        
        if pred_token == trg_field.vocab.stoi[trg_field.eos_token]:
            break
        
    trg_tokens = [trg_field.vocab.itos[i] for i in trg_indexes]
    
    return trg_tokens[1:]

In [51]:
example_idx = 10

src = vars(test_data.examples[example_idx])['src']
trg = vars(test_data.examples[example_idx])['trg']

print(f'소스 문장: {src}')
print(f'타겟 문장: {trg}')

소스 문장: ['.', 'freien', 'im', 'tag', 'schönen', 'einen', 'genießen', 'sohn', 'kleiner', 'ihr', 'und', 'mutter', 'eine']
타겟 문장: ['a', 'mother', 'and', 'her', 'young', 'song', 'enjoying', 'a', 'beautiful', 'day', 'outside', '.']


In [52]:
src = tokenize_de("Guten Abend.")

print(f'소스 문장: {src}')


소스 문장: ['.', 'Abend', 'Guten']
